In [1]:
from src.utils.extract_utils import gather_activations

/root/activation-additions-large-models/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch, transformers, accelerate

In [3]:
from src.utils.model_utils import load_gpt_model_and_tokeniser

In [4]:
model, tokenizer, MODEL_CONFIG = load_gpt_model_and_tokeniser(model_name="gpt2-xl")

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [5]:
td = gather_activations(
    "Good day sir, how are you doing today?",
    MODEL_CONFIG['attn_hook_names'],
    model,
    tokenizer,
)

In [11]:
td.keys()

odict_keys(['transformer.h.0.attn.c_proj', 'transformer.h.1.attn.c_proj', 'transformer.h.2.attn.c_proj', 'transformer.h.3.attn.c_proj', 'transformer.h.4.attn.c_proj', 'transformer.h.5.attn.c_proj', 'transformer.h.6.attn.c_proj', 'transformer.h.7.attn.c_proj', 'transformer.h.8.attn.c_proj', 'transformer.h.9.attn.c_proj', 'transformer.h.10.attn.c_proj', 'transformer.h.11.attn.c_proj', 'transformer.h.12.attn.c_proj', 'transformer.h.13.attn.c_proj', 'transformer.h.14.attn.c_proj', 'transformer.h.15.attn.c_proj', 'transformer.h.16.attn.c_proj', 'transformer.h.17.attn.c_proj', 'transformer.h.18.attn.c_proj', 'transformer.h.19.attn.c_proj', 'transformer.h.20.attn.c_proj', 'transformer.h.21.attn.c_proj', 'transformer.h.22.attn.c_proj', 'transformer.h.23.attn.c_proj', 'transformer.h.24.attn.c_proj', 'transformer.h.25.attn.c_proj', 'transformer.h.26.attn.c_proj', 'transformer.h.27.attn.c_proj', 'transformer.h.28.attn.c_proj', 'transformer.h.29.attn.c_proj', 'transformer.h.30.attn.c_proj', 'trans

In [9]:
MODEL_CONFIG.keys()

dict_keys(['n_heads', 'n_layers', 'resid_dim', 'name_or_path', 'attn_hook_names', 'layer_hook_names'])

In [6]:
act_tensors = [td[layer].input for layer in MODEL_CONFIG['attn_hook_names']]

In [7]:

act_tensors[0].shape

torch.Size([1, 10, 1600])

In [10]:
type(model)

transformers.models.gpt2.modeling_gpt2.GPT2LMHeadModel

In [11]:
MODEL_CONFIG['attn_hook_names']


['transformer.h.0.attn.c_proj',
 'transformer.h.1.attn.c_proj',
 'transformer.h.2.attn.c_proj',
 'transformer.h.3.attn.c_proj',
 'transformer.h.4.attn.c_proj',
 'transformer.h.5.attn.c_proj',
 'transformer.h.6.attn.c_proj',
 'transformer.h.7.attn.c_proj',
 'transformer.h.8.attn.c_proj',
 'transformer.h.9.attn.c_proj',
 'transformer.h.10.attn.c_proj',
 'transformer.h.11.attn.c_proj',
 'transformer.h.12.attn.c_proj',
 'transformer.h.13.attn.c_proj',
 'transformer.h.14.attn.c_proj',
 'transformer.h.15.attn.c_proj',
 'transformer.h.16.attn.c_proj',
 'transformer.h.17.attn.c_proj',
 'transformer.h.18.attn.c_proj',
 'transformer.h.19.attn.c_proj',
 'transformer.h.20.attn.c_proj',
 'transformer.h.21.attn.c_proj',
 'transformer.h.22.attn.c_proj',
 'transformer.h.23.attn.c_proj',
 'transformer.h.24.attn.c_proj',
 'transformer.h.25.attn.c_proj',
 'transformer.h.26.attn.c_proj',
 'transformer.h.27.attn.c_proj',
 'transformer.h.28.attn.c_proj',
 'transformer.h.29.attn.c_proj',
 'transformer.h.30.a